In [ ]:
import random #libreria para importar funciones relacionadas con numeros aleatorios

poblacion_size = 100 #tamaño de la poblacion en cada generacion del AG
max_generations = 100 #numero maximo de generaciones
mutation_rate = 0.1 #indica la probabilidad de que un gen mute

num_courses = 5 #numero de materias
num_rooms = 3 #numero de aulas
num_timeslots = 4 #indica cuantos intervalos de tiempo existen para programar los cursos

'''lista bidimensional con (num_rooms) filas y (num_timeslots) columnas,
cada celda de la matriz representa la combinacion de aula y horario,
todas las celdas tienen el valor de 1, todas las combinaciones son permitidas'''
constraints_matrix = [[1 for _ in range(num_timeslots)] for _ in range(num_rooms)]

#funcion fitness
def fitness(schedule):
  conflicts = 0 #numero de conflictos en el horario
  '''iteramos sobre todas las combinaciones posibles de aulas y horarios en el
  horario generado'''
  for room in range(num_rooms):
    for timeslot in range(num_timeslots):
      course = schedule[room][timeslot] #materia asiganada a la combinacion de aula y horario
      '''condiciones para considerar un conflicto:
      1.la celda correspondientes en la matriz no es igual a 1.
      2.la misma materia esta asignada en otra celda dentro del mismo aula. Se cuenta la cantidad de veces
      que la materia aparece en la lista de cursos asignados a ese aula, si el resultado es mayor que 1,
      significa que hay mas de una asignacion de la misma meteria en el mismo aula '''
      if constraints_matrix[room][timeslot] != 1 or schedule[room].count(course) > 1:
        conflicts += 1
  '''calculamos el fitness del horario dividiendo 1 entre la cantidad de conflictos mas 1,
  esto se realiza para asignar un mayor fitness a los horarios con menos conflictos'''
  return 1 / (conflicts + 1) # Cuanto menor sea el número de conflictos, mayor será el fitness

#generar la poblacion inicial
def generate_initial_poblacion():
  poblacion = []
  #bucle para generar el numero deseado de horarios en la poblacion inicial. cada iteracion genera un horario individual
  for _ in range(poblacion_size):
    '''representamos un horario individual con (schedule), con for asignamos materias en el horario,
    generamos un numero aleatorio utilizando (random.randit), este numero aleatorio representa la materia asignada
    a esa celda en particular, el rango es de 1 a 10, lo que indica que cada celda puede contener un curso aleatorio
    de entre 1 y 10'''
    schedule = [[random.randint(1, num_courses) for _ in range(num_timeslots)] for _ in range(num_rooms)]
    poblacion.append(schedule) #agregamos el horario a la lista poblacion
  return poblacion

#seleccionar individuos para reproduccion (torneo binario)
def selection(poblacion):
  selected = []
  for _ in range(poblacion_size):  #cada iteracion seleccionara un individuo
    candidates = random.sample(poblacion, 2) #seleccionamos 2 individuos de la poblacion al azar
    fitnesses = [fitness(candidate) for candidate in candidates] #aqui guardamos el fitness de cada individuo
    selected.append(candidates[fitnesses.index(max(fitnesses))]) #encontramos el valor maximo de fitness entre los individuos
  return selected

#realizamos la recombinacion de dos horarios
def crossover(schedule1, schedule2):
  child = []
  for i in range(num_rooms): #iteramos a traves de aulas en el horario.
    timeslots = [] #aqui almacenamos horarios de las materias para un aula especifico
    for j in range(num_timeslots): #iteramos a traves de timeslots en el horario
      '''utilizamos if para decidir que horaio tomar, la decision se toma de manera aleatoria con una
      probabilidad de 50% utilizando( random.random) < 0.5. Si el resultado es verdadero, se toma el resultado
      de schedule1 de lo contrario se toma el de schedule2'''
      timeslots.append(schedule1[i][j] if random.random() < 0.5 else schedule2[i][j])
    child.append(timeslots) #agregamos timesot a la lista child, lo que representa el horario del hijo para ese aula
  return child

#aplicar mutacion a un horario
def mutate(schedule):
  for i in range(num_rooms): #iteramos sobre el numero de aulas
    for j in range(num_timeslots): #iteramos a traves de timeslots
      '''preguntamos si se debe aplicar mutacion en la materia correspondiente al timeslot actual. la decision
      se toma aleatoriamente'''
      if random.random() < mutation_rate: #(mutation_rate)tasa de mutacion
        '''si if es verdadero, se genera aleatoriamente un nuevo valor de materia y se asigna a la materia del
        horario y timeslot correspondiente'''
        schedule[i][j] = random.randint(1, num_courses)

#AG principal
def genetic_algorithm():
  poblacion = generate_initial_poblacion() #iniciamos la poblacion con horarios generados aleatoriamente
  for _ in range(max_generations): #iteramos a traves de numero de generaciones
    poblacion = selection(poblacion) #realizamos la seleccion de poblacion, utilizando los mejores horarios basado en el fitness
    new_poblacion = [] #cremos nueva poblacion mediante la combinacion de los horarios seleccionados.
    '''en cada iteracion, se seleccionan aleatoriamente dos padres de la poblacion actual
    (random.sample(poblacion, 2)), luego realizamos el cruce (crossover) entre los padres para generar dos hijos.
    los hijos tambien sufren mutacion. finalmente los descendientes se agregan a la poblacion. '''
    for _ in range(poblacion_size //2): #division entera
      parent1, parent2 = random.sample(poblacion, 2)
      child1 = crossover(parent1, parent2)
      child2 = crossover(parent2, parent1)
      mutate(child1)
      mutate(child2)
      new_poblacion.extend([child1, child2])
    poblacion = new_poblacion #la nueva poblacion reemplaza a la poblacion actual

  best_schedule = max(poblacion, key=fitness) #encontramos el mejor horario. devuelve el horario con el fitness mas alto
  best_fitness = fitness(best_schedule) #calculamos el fitness del mejor horario encontrado

  return best_schedule, best_fitness

#ejecuta el algoritmo y obtiene el mejor horario y su fitness, almacenandolos en las variavles para su posterior uso
best_schedule, best_fitness = genetic_algorithm()

#mostramos en pantalla el mejor horario encontrado y su fitness
print("El mejor hoario encontrado es:")
for i in range(num_rooms):
  for j in range(num_timeslots):
    print(best_schedule[i][j], end='\t')
  print()
print("Fitness del horario: ", best_fitness)




El mejor hoario encontrado es:
2	5	1	3	
2	4	1	3	
4	5	1	3	
Fitness del horario:  1.0
